# Lösungsvorschlag Aufgabe 32

In [1]:
using LinearAlgebra
using ForwardDiff     

### Relaxierte Newton-Iteration

In [2]:
macro stop(name, test)                                     
    return esc(:(if ($test) info = $name; break; end))
end

function NewtonRelax(f, x; tol=1e-14, max_steps=25, λ_min=1e-6)
    y, λ, steps, info = f(x), 1., 0, Nothing
    while info == Nothing
        @stop(:MAXSTEPS, steps >= max_steps) 
        steps += 1
        J = lu!(ForwardDiff.jacobian(f, x), check=false) 
        @stop(:JSING, !issuccess(J))
        Δx = J\y
        Δx_norm = norm(Δx)
        @stop(:CONV, Δx_norm <= tol)
        x₀ = x
        while true
            x = x₀ - λ*Δx
            y = f(x)
            Δx̄ = J\y
            if (norm(Δx̄) <= (1-λ/2)*Δx_norm) 
                λ = min(1.0, 2λ); break; 
            end
            λ = λ/2
            @stop(:NOCONV, λ < λ_min)
        end
    end
    
    return x, steps, info
end;

## Formel für Volumen

In [3]:
# z = [x, y]; s und h eliminiert
V(z) = ((x,y) = z; π/3*(x^2+x*y+y^2)*sqrt((1-x^2)^2/(x+y)^2-(y-x)^2))

V (generic function with 1 method)

### Vereinfachte Geometrie mit Zylinder

In [4]:
# Zylinder mit x = y
# z = [x]
VZylinder(z) = V([z[1], z[1]])
∇VZylinder(z) = ForwardDiff.gradient(VZylinder, z)

z_zylinder, = NewtonRelax(∇VZylinder, [2.]) # jeder Startwert 0 < x ≠ 1 taugt

([0.5773502691896257], 7, :CONV)

### Eimer

In [5]:
# z = [x, y] 
∇V(z) = ForwardDiff.gradient(V, z)

z₀ = [1.0, 1.0]*z_zylinder[1] # Startwert aus Zylinder-Geometrie
z_Eimer, = NewtonRelax(∇V, z₀)

([0.40337827902317364, 0.7353084448018562], 7, :CONV)

In [6]:
@show V(z₀) 
@show V(z_Eimer)
@show Volumen_Gewinn_in_Prozent=round(100(V(z_Eimer)/V(z₀)-1), digits=1)

V(z₀) = 0.6045997880780727
V(z_Eimer) = 0.6870931783105633
Volumen_Gewinn_in_Prozent = round(100 * (V(z_Eimer) / V(z₀) - 1), digits = 1) = 13.6


13.6